# Geodata Hacks

## Massage Swisstopo Boundary Data

`data/towns.geojson` contains the polygons for Swiss admin entities such as `Gemeinde`, `Bezirk`, `Kanton` etc, but we need to massage the dataset a bit to be compatible with the TG datasets.

### Steps 1: Import the data and have a look

In [ ]:
import geopandas
municipalities = geopandas.read_file("../data/towns.geojson")
municipalities


Ok, so we've read the swisstopo database, but there are many fields we don't want. Let's clean up:

In [11]:
import pandas as pd

# Fix up municipalities by parsing the bfs_nr from the gml_id and converting it
# to int64.

# 1: create a series where we extract the integer number from the 'AdministrativeUnit_XY_1234' 
# string in gml_id, where 'XY' is one of 'Gemeinde', 'Bezirk', 'Kanton'.
bfs_nrs = municipalities['gml_id'].str.extract(r"(\d+)")
# Add the series to the dataset after converting it to numeric.
municipalities['bfs_nr_gemeinde'] = pd.to_numeric(bfs_nrs[0])

# Let's only retain what we need: BFS nr, geometry, type, country
municipalities = municipalities[['bfs_nr_gemeinde', 'nationalCode', 'LocalisedCharacterString', 'text', 'geometry']]

# There are four rows without BFS number
missing_number = municipalities[municipalities['bfs_nr_gemeinde'].isna()]
print(missing_number)
# Ah, the country-level entries miss the number - let's drop these:
municipalities = municipalities.dropna(subset='bfs_nr_gemeinde')

# Now we can cast the BFS nr to int64 to make it compatible with the TG energy dataset.
municipalities['bfs_nr_gemeinde'].astype('int64')

# Finally, rename the columns to be more descriptive
municipalities = municipalities.rename(columns={'nationalCode' : 'country', 'LocalisedCharacterString': 'type', 'text' : 'name'})

municipalities


      bfs_nr_gemeinde nationalCode LocalisedCharacterString           text  \
577               NaN           CH                     Land        Schweiz   
648               NaN           LI                     Land  Liechtenstein   
1706              NaN           IT                     Land         Italia   
2085              NaN           DE                     Land    Deutschland   

                                               geometry  
577   MULTIPOLYGON (((708720.565 5845960.697, 707286...  
648   MULTIPOLYGON (((1066328.311 5973986.047, 10663...  
1706  MULTIPOLYGON (((998369.170 5774248.479, 998372...  
2085  MULTIPOLYGON (((968716.436 6055745.395, 968701...  


,bfs_nr_gemeinde,country,type,name,geometry
0,3762.0,CH,Gemeinde,Scuol,"MULTIPOLYGON (((1139077.532 5921141.517, 11400..."
1,3575.0,CH,Gemeinde,Laax,"MULTIPOLYGON (((1025590.950 5924456.089, 10256..."
2,5529.0,CH,Gemeinde,Oulens-sous-Echallens,"MULTIPOLYGON (((732842.653 5882076.703, 732766..."
3,3316.0,CH,Gemeinde,Weesen,"MULTIPOLYGON (((1009859.085 5963319.102, 10098..."
4,906.0,CH,Gemeinde,Schangnau,"MULTIPOLYGON (((885081.295 5909434.189, 885096..."
...,...,...,...,...,...
2321,2305.0,CH,Bezirk,Hérens,"MULTIPOLYGON (((820225.037 5808928.463, 820415..."
2322,4181.0,CH,Gemeinde,Wittnau,"MULTIPOLYGON (((889975.343 6021115.485, 889971..."
2323,5761.0,CH,Gemeinde,Romainmôtier-Envy,"MULTIPOLYGON (((721847.754 5888985.938, 721852..."
2324,442.0,CH,Gemeinde,Romont (BE),"MULTIPOLYGON (((814775.095 5977288.045, 814849..."


### Save
Re-export the reduced dataset

In [19]:
municipalities.to_file('../data/towns_fixed.geojson', driver='GeoJSON')
